In [ ]:
import pandas as pd
import amp_pd_peptide
import numpy as np
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from pandas_profiling import ProfileReport
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
import matplotlib.pyplot as plt
import copy
from keras.layers import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM, GRU
from tensorflow import keras
import time


In [ ]:
clinical = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")
print(clinical.shape)
clinical.head(2)

In [ ]:
clinical.isnull().sum()

In [ ]:
clinical['updrs_1'].fillna((clinical['updrs_1'].mean()), inplace=True)
clinical['updrs_2'].fillna((clinical['updrs_2'].mean()), inplace=True)
clinical['updrs_3'].fillna((clinical['updrs_3'].mean()), inplace=True)
clinical.drop(['upd23b_clinical_state_on_medication'],axis = 1, inplace = True)
clinical['updrs_4'].fillna(0, inplace=True)
clinical.isnull().sum()

In [ ]:
proteins = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv")
print(proteins.shape)
print(proteins.isnull().sum().sum())
proteins.head(2)

In [ ]:
peptides = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv")
print(peptides.shape)
print(peptides.isnull().sum().sum())
peptides.head(2)

In [ ]:
def cycle_cehck(all_merge):
    temp_cycle = []
    same_cycle = 1
    for i, j in all_merge.iterrows():
        if i == 0:
            prev_visit_id = j.visit_id
            prev_patient_id = j.patient_id
            prev_visit_month = j.visit_month
            same_cycle = 1
            temp_cycle.append(same_cycle)
            continue 
    
    
        if((prev_patient_id == j.patient_id) and (prev_visit_month == j.visit_month)):
            temp_cycle.append(same_cycle)
        elif((prev_patient_id == j.patient_id) and (prev_visit_month != j.visit_month)):
            prev_visit_id = j.visit_id
            prev_patient_id = j.patient_id
            prev_visit_month = j.visit_month
            same_cycle += 1
            temp_cycle.append(same_cycle)
        else:
            prev_visit_id = j.visit_id
            prev_patient_id = j.patient_id
            prev_visit_month = j.visit_month
            same_cycle = 1
            temp_cycle.append(same_cycle)
    return temp_cycle

In [ ]:
# 为以上所有内容编写一个函数，使用 same 进行测试，并将测试数据视为 train。
# 启动 LSTM

In [ ]:
enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=999)
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
def data_prep(clinical, peptides, proteins, train=1):
    
    #proteins_peptides = proteins.merge(peptides, how='inner', on=['visit_id','visit_month','patient_id','UniProt'])
    #all_merge = clinical.merge(proteins_peptides, how='left', on=['visit_id', 'visit_month', 'patient_id'])
    
    all_merge = clinical.merge(proteins, how='left', on=['visit_id', 'visit_month', 'patient_id'])

    all_merge['UniProt'].fillna("other", inplace=True)
    all_merge['NPX'].fillna((all_merge['NPX'].mean()), inplace=True)
    #all_merge['Peptide'].fillna("other", inplace=True)
    #all_merge['PeptideAbundance'].fillna((all_merge['PeptideAbundance'].mean()), inplace=True)
    '''
    if(train):
        all_merge[["UniProt","Peptide"]] =\
        enc.fit_transform(all_merge[["UniProt","Peptide"]])
    else :
        all_merge[["UniProt","Peptide"]] =\
        enc.transform(all_merge[["UniProt","Peptide"]])
    '''
    if(train):
        all_merge[["UniProt"]] =\
        enc.fit_transform(all_merge[["UniProt"]])
    else :
        all_merge[["UniProt"]] =\
        enc.transform(all_merge[["UniProt"]])

        
    temp_cycle = cycle_cehck(all_merge)
    all_merge['cycle'] = temp_cycle
    
    if(train):
        #all_merge.drop(['visit_id', 'patient_id'],axis = 1, inplace = True)
        cols_normalize = all_merge.columns.difference(['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id'])
        norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    else :
        all_merge['month_raw'] =  copy.deepcopy(all_merge.loc[:, 'visit_month'])
        cols_normalize = all_merge.columns.difference(['visit_id','patient_id','updrs_test','row_id','month_raw'])
        norm_train_df = pd.DataFrame(min_max_scaler.transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['visit_id','patient_id','updrs_test','row_id','month_raw']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    
    return all_merge

In [ ]:
data_prep_train = data_prep(clinical, peptides, proteins, train=1)
data_prep_train.head(5)

In [ ]:
# test and submission.

submission = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv")

test_proteins = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv")
test_proteins.drop(['group_key'],axis = 1, inplace = True)

test_peptides = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv")
test_peptides.drop(['group_key'],axis = 1, inplace = True)

test_clinical = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv")
test_clinical.drop(['group_key'],axis = 1, inplace = True)

data_prep_test = data_prep(test_clinical, test_peptides, test_proteins, train=0)
print(data_prep_test.shape)
data_prep_test.head(5)

In [ ]:
test_clinical.head(2)

In [ ]:
sequence_length = 50
def sequence_generator(feature_df, seq_length, seq_cols):
    if(feature_df.shape[0] < sequence_length):
        feature_df = pd.concat([feature_df, pd.concat([feature_df[-1:]]*(sequence_length+1 - feature_df.shape[0]))])
    feature_array = feature_df[seq_cols].values
    num_elements = feature_array.shape[0]    
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield feature_array[start:stop, :]

In [ ]:
def label_generator(label_df, seq_length, label):
    if(label_df.shape[0] < sequence_length):
        label_df = pd.concat([label_df, pd.concat([label_df[-1:]]*(sequence_length+1 - label_df.shape[0]))])
    label_array = label_df[label].values
    num_elements = label_array.shape[0]    
    return label_array[seq_length:num_elements, :]

In [ ]:
#seq_cols = ['visit_month','UniProt','NPX','Peptide','PeptideAbundance','cycle']
seq_cols = ['visit_month','UniProt','NPX','cycle']

In [ ]:
def train_sed_gen(data_prep_train):
    seq_gen_train = (list(sequence_generator(data_prep_train[data_prep_train['patient_id'] == patient_id], sequence_length, seq_cols))
           for patient_id in data_prep_train['patient_id'].unique())

    #print(list(seq_gen))
    seq_gen_train = [ele for ele in seq_gen_train if ele != []]

    seq_gen_train_f = np.concatenate(list(seq_gen_train)).astype(np.float32)
    return seq_gen_train_f

In [ ]:
seq_gen_train_f = train_sed_gen(data_prep_train)
seq_gen_train_f.shape

In [ ]:
def train_label_gen(data_prep_train, target):
    label_gen = [label_generator(data_prep_train[data_prep_train['patient_id'] == patient_id], sequence_length, [target]) 
             for patient_id in data_prep_train['patient_id'].unique()]
    label_set = np.concatenate(label_gen).astype(np.float32)
    return label_set

In [ ]:
target_updrs_1 = train_label_gen(data_prep_train, 'updrs_1')
print(target_updrs_1.shape)
target_updrs_2 = train_label_gen(data_prep_train, 'updrs_2')
print(target_updrs_2.shape)
target_updrs_3 = train_label_gen(data_prep_train, 'updrs_3')
print(target_updrs_3.shape)
target_updrs_4 = train_label_gen(data_prep_train, 'updrs_4')
print(target_updrs_4.shape)

In [ ]:
data_prep_test['row_id'].unique()

In [ ]:
# test seq
def test_seq_gen(data_prep_test):
    seq_gen_test = (list(sequence_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, seq_cols))
           for row_id in data_prep_test['row_id'].unique())

    #print(list(seq_gen))
    seq_gen_test = [ele for ele in seq_gen_test if ele != []]

    seq_gen_test_f = np.concatenate(list(seq_gen_test)).astype(np.float32)
    return seq_gen_test_f

In [ ]:
seq_gen_test_f = test_seq_gen(data_prep_test)
seq_gen_test_f.shape

In [ ]:
def test_data_save(data_prep_test):
    row_id_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['row_id']) 
             for row_id in data_prep_test['row_id'].unique()]
    row_id_out = np.concatenate(row_id_out)
    visit_month_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['month_raw']) 
             for row_id in data_prep_test['row_id'].unique()]
    visit_month_out = np.concatenate(visit_month_out)
    visit_id_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['visit_id']) 
             for row_id in data_prep_test['row_id'].unique()]
    visit_id_out = np.concatenate(visit_id_out)
    updrs_test_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['updrs_test']) 
             for row_id in data_prep_test['row_id'].unique()]
    updrs_test_out = np.concatenate(updrs_test_out)
    
    patient_id_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['patient_id']) 
             for row_id in data_prep_test['row_id'].unique()]
    patient_id_out = np.concatenate(patient_id_out)


    row_id_out_df = pd.DataFrame(row_id_out, columns = ['row_id'])
    visit_month_out_df = pd.DataFrame(visit_month_out, columns = ['visit_month'])
    visit_id_out_df = pd.DataFrame(visit_id_out, columns = ['visit_id'])
    updrs_test_out_df = pd.DataFrame(updrs_test_out, columns = ['updrs_test'])
    patient_id_out_df = pd.DataFrame(patient_id_out, columns = ['patient_id'])
    pred_df = pd.concat([visit_id_out_df, visit_month_out_df,patient_id_out_df,updrs_test_out_df,row_id_out_df], axis=1)
    return pred_df

In [ ]:
pred_df = test_data_save(data_prep_test)
print(pred_df.shape)
pred_df.head(2)

In [ ]:
pred_df.groupby(['row_id', 'visit_month','visit_id'])['row_id', 'visit_month','visit_id'].size()

In [ ]:
import tensorflow.keras.backend as K
def smape_loss(y_true, y_pred):
    epsilon = 0.1
    summ = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = K.abs(y_pred - y_true) / summ * 2.0
    return smape

In [ ]:
def plot_model_accuracy(model_updrs_1_name_history, width = 10, height = 10):
  fig_acc = plt.figure(figsize=(width, height))
  plt.plot(model_updrs_1_name_history.history['mean_absolute_error'])
  plt.plot(model_updrs_1_name_history.history['val_mean_absolute_error'])
  plt.title('model')
  plt.ylabel('mse')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
#plot_model_accuracy(history_updrs_1, 5, 5)

In [ ]:
# building model_updrs_1 for updrs_1
features_dim = seq_gen_train_f.shape[2] # number of features
out_dim = target_updrs_1.shape[1] 
#print(target_updrs_1[:5])
print("working on model_updrs_1")
print("Features dimension: ", features_dim)
print("Output dimension: ", out_dim)

model_updrs_1 = Sequential()

# Understanding of return_sequences and connection across LSTM layers
model_updrs_1.add(LSTM(
input_shape=(sequence_length, features_dim),
        units=100,
         return_sequences=True))
model_updrs_1.add(Dropout(0.2))

model_updrs_1.add(LSTM(
          units=50,
          return_sequences=False))
model_updrs_1.add(Dropout(0.2))

    # output layer
model_updrs_1.add(Dense(1))
model_updrs_1.add(Activation("linear"))

    # Compile the model_updrs_1
model_updrs_1.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

    #print(model_updrs_1.summary())

    # Define path to save model_updrs_1
model_updrs_1_path_updrs_1 = "/kaggle/working/updrs_1_binary_model_updrs_1.h5"
    
epochs = 30
batch_size = 100
start = time.time()
    # Fit the network
history_updrs_1 = model_updrs_1.fit(seq_gen_train_f, target_updrs_1, epochs=epochs, batch_size=batch_size, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_updrs_1_path_updrs_1, monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )
end = time.time()
print("Total time taken for training: ", "{:.2f}".format((end-start)), " secs")
train_predict_updrs_1 = model_updrs_1.predict(seq_gen_train_f)
print(train_predict_updrs_1.shape)
print(train_predict_updrs_1)
test_predict_updrs_1 = model_updrs_1.predict(seq_gen_test_f)
print(test_predict_updrs_1.shape)
print(test_predict_updrs_1)
plot_model_accuracy(history_updrs_1, 5, 5)

In [ ]:
# building model_updrs_2 for updrs_2
print("working on model_updrs_2")
features_dim = seq_gen_train_f.shape[2] # number of features
out_dim = target_updrs_2.shape[1] 
#print(target_updrs_2[:5])

print("Features dimension: ", features_dim)
print("Output dimension: ", out_dim)

model_updrs_2 = Sequential()

# Understanding of return_sequences and connection across LSTM layers
model_updrs_2.add(LSTM(
input_shape=(sequence_length, features_dim),
        units=100,
         return_sequences=True))
model_updrs_2.add(Dropout(0.2))

model_updrs_2.add(LSTM(
          units=50,
          return_sequences=False))
model_updrs_2.add(Dropout(0.2))

    # output layer
model_updrs_2.add(Dense(1))
model_updrs_2.add(Activation("linear"))

model_updrs_2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

#print(model_updrs_2.summary())

model_updrs_2_path_updrs_2 = "/kaggle/working/updrs2_binary_model_updrs_2.h5"
    
epochs = 30
batch_size = 100
start = time.time()
    # Fit the network
history_updrs_2 = model_updrs_2.fit(seq_gen_train_f, target_updrs_2, epochs=epochs, batch_size=batch_size, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_updrs_2_path_updrs_2, monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )
end = time.time()
print("Total time taken for training: ", "{:.2f}".format((end-start)), " secs")
train_predict_updrs_2 = model_updrs_2.predict(seq_gen_train_f)
print(train_predict_updrs_2.shape)
print(train_predict_updrs_2)
test_predict_updrs_2 = model_updrs_2.predict(seq_gen_test_f)
print(test_predict_updrs_2.shape)
print(test_predict_updrs_2)
plot_model_accuracy(history_updrs_2, 5, 5)

In [ ]:
print("working on model_updrs_3")
# building model_updrs_3 for updrs_3
features_dim = seq_gen_train_f.shape[2] # number of features
out_dim = target_updrs_3.shape[1] 
#print(target_updrs_3[:5])

print("Features dimension: ", features_dim)
print("Output dimension: ", out_dim)

model_updrs_3 = Sequential()

# Understanding of return_sequences and connection across LSTM layers
model_updrs_3.add(LSTM(
input_shape=(sequence_length, features_dim),
        units=100,
         return_sequences=True))
model_updrs_3.add(Dropout(0.2))

model_updrs_3.add(LSTM(
          units=50,
          return_sequences=False))
model_updrs_3.add(Dropout(0.2))

    # output layer
model_updrs_3.add(Dense(1))
model_updrs_3.add(Activation("linear"))

model_updrs_3.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

#print(model_updrs_3.summary())

model_updrs_3_path_updrs_3 = "/kaggle/working/updrs3_binary_model_updrs_3.h5"
    
epochs = 30
batch_size = 100
start = time.time()
    # Fit the network
history_updrs_3 = model_updrs_3.fit(seq_gen_train_f, target_updrs_3, epochs=epochs, batch_size=batch_size, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_updrs_3_path_updrs_3, monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )
end = time.time()
print("Total time taken for training: ", "{:.2f}".format((end-start)), " secs")
train_predict_updrs_3 = model_updrs_3.predict(seq_gen_train_f)
print(train_predict_updrs_3.shape)
print(train_predict_updrs_3)
test_predict_updrs_3 = model_updrs_3.predict(seq_gen_test_f)
print(test_predict_updrs_3.shape)
print(test_predict_updrs_3)
plot_model_accuracy(history_updrs_3, 5, 5)

In [ ]:
# building model_updrs_4 for updrs_4
print("working on model_updrs_4")
features_dim = seq_gen_train_f.shape[2] # number of features
out_dim = target_updrs_4.shape[1] 
#print(target_updrs_4[:5])

print("Features dimension: ", features_dim)
print("Output dimension: ", out_dim)

model_updrs_4 = Sequential()

# Understanding of return_sequences and connection across LSTM layers
model_updrs_4.add(LSTM(
input_shape=(sequence_length, features_dim),
        units=100,
         return_sequences=True))
model_updrs_4.add(Dropout(0.2))

model_updrs_4.add(LSTM(
          units=50,
          return_sequences=False))
model_updrs_4.add(Dropout(0.2))

    # output layer
model_updrs_4.add(Dense(1))
model_updrs_4.add(Activation("linear"))

model_updrs_4.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

#print(model_updrs_3.summary())

model_updrs_4_path_updrs_4 = "/kaggle/working/updrs4_binary_model_updrs_4.h5"
    
epochs = 30
batch_size = 100
start = time.time()
    # Fit the network
history_updrs_4 = model_updrs_4.fit(seq_gen_train_f, target_updrs_4, epochs=epochs, batch_size=batch_size, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_updrs_4_path_updrs_4, monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )
end = time.time()
print("Total time taken for training: ", "{:.2f}".format((end-start)), " secs")
train_predict_updrs_4 = model_updrs_4.predict(seq_gen_train_f)
print(train_predict_updrs_4.shape)
print(train_predict_updrs_4)
test_predict_updrs_4 = model_updrs_4.predict(seq_gen_test_f)
print(test_predict_updrs_4.shape)
print(test_predict_updrs_4)
plot_model_accuracy(history_updrs_4, 5, 5)

In [ ]:
pred_df['prediction_updrs_1'] = test_predict_updrs_1
pred_df['prediction_updrs_2'] = test_predict_updrs_2
pred_df['prediction_updrs_3'] = test_predict_updrs_3
pred_df['prediction_updrs_4'] = test_predict_updrs_4
pred_df.head(5)

In [ ]:
final_predicts = pred_df.groupby(['row_id', 'visit_month','visit_id','updrs_test'])\
['prediction_updrs_1','prediction_updrs_2','prediction_updrs_3', 'prediction_updrs_4'].mean().reset_index()
final_predicts

In [ ]:
def map_test(x):
    updrs = x.split('_')[2] + '_' + x.split('_')[3]
    #print(updrs)
    month = int(x.split('_plus_')[1].split('_')[0])
    #print(month)
    visit_id = x.split('_')[0] + '_' + x.split('_')[1]
    #print(visit_id)
    if updrs=='updrs_1':
        check = df[(df.visit_id == visit_id) & (df.visit_month == month)]['prediction_updrs_1']
        if(check.shape[0] == 0):
            rating = df[df.visit_id == visit_id]['prediction_updrs_1'].values[0]
        else : 
            rating = check.values[0]
    elif updrs=='updrs_2':
        check = df[(df.visit_id == visit_id) & (df.visit_month == month)]['prediction_updrs_2']
        if(check.shape[0] == 0):
            rating = df[df.visit_id == visit_id]['prediction_updrs_2'].values[0]
        else : 
            rating = check.values[0]
    elif updrs =='updrs_3':
        check = df[(df.visit_id == visit_id) & (df.visit_month == month)]['prediction_updrs_3']
        if(check.shape[0] == 0):
            rating = df[df.visit_id == visit_id]['prediction_updrs_3'].values[0]
        else : 
            rating = check.values[0]
    elif updrs =='updrs_4':
        check = df[(df.visit_id == visit_id) & (df.visit_month == month)]['prediction_updrs_4']
        if(check.shape[0] == 0):
            rating = df[df.visit_id == visit_id]['prediction_updrs_4'].values[0]
        else : 
            rating = check.values[0]
    return rating

In [ ]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()
iter_test = env.iter_test()

In [ ]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    
    data_prep_test = data_prep(test, test_peptides, test_proteins, train=0)
    data_prep_test = data_prep_test.fillna(0)
    #display(data_prep_test)
    test_seq_gen_test_f = test_seq_gen(data_prep_test)
    print(test_seq_gen_test_f.shape)
    pred_df_test = test_data_save(data_prep_test)
    print(pred_df_test.shape)
    #display(pred_df_test)
    test_updrs_1 = model_updrs_1.predict(test_seq_gen_test_f)
    test_updrs_2 = model_updrs_2.predict(test_seq_gen_test_f)
    test_updrs_3 = model_updrs_3.predict(test_seq_gen_test_f)
    test_updrs_4 = model_updrs_4.predict(test_seq_gen_test_f)
    
    pred_df_test['prediction_updrs_1'] = test_updrs_1
    pred_df_test['prediction_updrs_2'] = test_updrs_2
    pred_df_test['prediction_updrs_3'] = test_updrs_3
    pred_df_test['prediction_updrs_4'] = test_updrs_4
    #display(pred_df_test)
    
    df = pred_df_test.groupby(['row_id', 'visit_month','visit_id','updrs_test'])\
    ['prediction_updrs_1','prediction_updrs_2','prediction_updrs_3', 'prediction_updrs_4'].mean().reset_index()
    
    sample_submission['rating'] = sample_submission['prediction_id'].apply(map_test)

    env.predict(sample_submission)